In [39]:
from transformers import DataCollatorWithPadding,AutoModelForSequenceClassification, Trainer, TrainingArguments,AutoTokenizer,AutoModel,AutoConfig
from transformers import default_data_collator
from transformers import Wav2Vec2FeatureExtractor, HubertForSequenceClassification
import torch
import torch.nn as nn
from datasets import Dataset, Audio, Value, Features,load_dataset,ClassLabel
from transformers import Wav2Vec2Processor
from transformers.modeling_outputs import SequenceClassifierOutput
import numpy as np
from transformers import AdamW,get_scheduler
from datasets import load_metric
from tqdm.auto import tqdm
import os
from torch.utils.data import DataLoader
x = [str(i) for i in range(0,100,1)]
features = Features(
    {
        "id": Value("string"),
        "speaker_id": Value("string"), 
        'path': Value('string'),
        "audio": Audio(sampling_rate=16000),
        "label": ClassLabel(num_classes=100,names=x,names_file=None,id=None),
    }
)


dataset = load_dataset('csv', data_files={'train': 'train_100.csv', 'dev': 'dev_100.csv', 'test': 'test_100.csv',},features=features)
dataset = dataset.map(remove_columns=(['path','speaker_id']),num_proc=24)
# dataset = load_dataset("hf-internal-testing/librispeech_asr_demo", "clean", split="validation")
dataset = dataset.sort("label")
sampling_rate = dataset["train"].features["audio"].sampling_rate
feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained("superb/hubert-base-superb-ks")
#feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained("facebook/wav2vec2-base")
# processor = Wav2Vec2Processor(feature_extractor=feature_extractor,tokenizer=None)

# model = HubertForSequenceClassification.from_pretrained("superb/hubert-base-superb-ks")

# # audio file is decoded on the fly
# inputs = feature_extractor(dataset[0]["audio"]["array"], sampling_rate=sampling_rate, return_tensors="pt")

# with torch.no_grad():
#     logits = model(**inputs).logits

# predicted_class_ids = torch.argmax(logits, dim=-1).item()
# predicted_label = model.config.id2label[predicted_class_ids]
# predicted_label

Using custom data configuration default-50c0c0358139fbe1
Found cached dataset csv (/storage/home/hcocice1/vprakash40/.cache/huggingface/datasets/csv/default-50c0c0358139fbe1/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)
100%|██████████| 3/3 [00:00<00:00, 359.12it/s]
Loading cached processed dataset at /storage/home/hcocice1/vprakash40/.cache/huggingface/datasets/csv/default-50c0c0358139fbe1/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-62b88ac5a716b157.arrow
Loading cached processed dataset at /storage/home/hcocice1/vprakash40/.cache/huggingface/datasets/csv/default-50c0c0358139fbe1/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-0c413296ec129391.arrow
Loading cached processed dataset at /storage/home/hcocice1/vprakash40/.cache/huggingface/datasets/csv/default-50c0c0358139fbe1/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-3416df98752e61a7.arrow
Loading cached processed d

In [40]:
dataset['train']

Dataset({
    features: ['id', 'audio', 'label'],
    num_rows: 11125
})

In [41]:
max_duration = 20.0  # seconds

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = "cpu"

def preprocess_function(examples):
    audio_arrays = [examples['audio'][i]['array'] for i in range(len(examples['id']))]
    #audio_arrays = [x["array"] for x in examples["audio"]]
    inputs = feature_extractor(
        audio_arrays, 
        sampling_rate=feature_extractor.sampling_rate, 
        max_length=int(feature_extractor.sampling_rate * max_duration), 
        truncation=True,
        padding='max_length'

    )
    return inputs

In [42]:
dataset['train']

Dataset({
    features: ['id', 'audio', 'label'],
    num_rows: 11125
})

In [43]:
preprocess_function(dataset['train'][:5])

{'input_values': array([[ 7.0251465e-02,  7.0281982e-02,  9.1552734e-02, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [-8.0871582e-03,  1.2512207e-03,  7.6599121e-03, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [-1.1596680e-03, -7.6293945e-04, -7.9345703e-04, ...,
         1.1596680e-03,  7.6293945e-04,  1.2512207e-03],
       [ 1.5869141e-03,  1.2207031e-03,  1.5563965e-03, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [ 3.0517578e-05, -6.1035156e-05,  0.0000000e+00, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00]], dtype=float32), 'attention_mask': [array([1, 1, 1, ..., 0, 0, 0], dtype=int32), array([1, 1, 1, ..., 0, 0, 0], dtype=int32), array([1, 1, 1, ..., 1, 1, 1], dtype=int32), array([1, 1, 1, ..., 0, 0, 0], dtype=int32), array([1, 1, 1, ..., 0, 0, 0], dtype=int32)]}

In [44]:
dataset = dataset.map(preprocess_function, remove_columns=["audio"], batched=True)
dataset

Loading cached processed dataset at /storage/home/hcocice1/vprakash40/.cache/huggingface/datasets/csv/default-50c0c0358139fbe1/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-a28780eb523e8ee9.arrow
Loading cached processed dataset at /storage/home/hcocice1/vprakash40/.cache/huggingface/datasets/csv/default-50c0c0358139fbe1/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-380d350aad4839b4.arrow
Loading cached processed dataset at /storage/home/hcocice1/vprakash40/.cache/huggingface/datasets/csv/default-50c0c0358139fbe1/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-56108a42e884fbaf.arrow


DatasetDict({
    train: Dataset({
        features: ['id', 'label', 'input_values', 'attention_mask'],
        num_rows: 11125
    })
    dev: Dataset({
        features: ['id', 'label', 'input_values', 'attention_mask'],
        num_rows: 533
    })
    test: Dataset({
        features: ['id', 'label', 'input_values', 'attention_mask'],
        num_rows: 632
    })
})

In [45]:
dataset.set_format("torch",columns=["id","input_values", "label"])
train_data_collator = default_data_collator
dev_data_collator = default_data_collator
test_data_collator = default_data_collator

train_dataloader = DataLoader(
    dataset["train"], shuffle=True, batch_size=32, collate_fn=train_data_collator
)
dev_dataloader = DataLoader(
    dataset["dev"], batch_size=32, collate_fn=dev_data_collator
)

In [46]:
#model_ft = HubertForSequenceClassification.from_pretrained("facebook/wav2vec2-base")
model_ft = HubertForSequenceClassification.from_pretrained("superb/hubert-base-superb-ks")

In [47]:
model_ft.eval()

HubertForSequenceClassification(
  (hubert): HubertModel(
    (feature_extractor): HubertFeatureEncoder(
      (conv_layers): ModuleList(
        (0): HubertGroupNormConvLayer(
          (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)
          (activation): GELUActivation()
          (layer_norm): GroupNorm(512, 512, eps=1e-05, affine=True)
        )
        (1): HubertNoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
          (activation): GELUActivation()
        )
        (2): HubertNoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
          (activation): GELUActivation()
        )
        (3): HubertNoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
          (activation): GELUActivation()
        )
        (4): HubertNoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False

In [48]:
dataset["test"]['label']

tensor([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  1,  1,  1,  1,  1,  1,  1,
         2,  2,  2,  2,  2,  3,  3,  3,  3,  3,  3,  3,  3,  4,  4,  4,  4,  4,
         4,  5,  5,  5,  5,  5,  5,  6,  6,  6,  6,  6,  6,  7,  7,  7,  7,  7,
         8,  8,  8,  8,  8,  8,  8,  9,  9,  9,  9,  9,  9, 10, 10, 10, 10, 10,
        11, 11, 11, 11, 11, 11, 12, 12, 12, 12, 12, 13, 13, 13, 13, 13, 13, 14,
        14, 14, 14, 14, 14, 15, 15, 15, 15, 15, 15, 15, 16, 16, 16, 16, 16, 16,
        17, 17, 17, 17, 17, 17, 17, 18, 18, 18, 18, 18, 18, 19, 19, 19, 19, 19,
        20, 20, 20, 20, 20, 21, 21, 21, 21, 21, 21, 22, 22, 22, 22, 22, 22, 23,
        23, 23, 23, 23, 23, 23, 24, 24, 24, 24, 24, 24, 25, 25, 25, 25, 25, 25,
        25, 25, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 27, 27, 27, 27,
        27, 27, 27, 28, 28, 28, 28, 28, 28, 28, 29, 29, 29, 29, 29, 29, 29, 30,
        30, 30, 30, 30, 30, 30, 30, 30, 31, 31, 31, 31, 31, 32, 32, 32, 32, 32,
        32, 32, 33, 33, 33, 33, 33, 33, 

In [49]:
metric_name="accuracy"
metric = load_metric(metric_name)
model_ft.eval()


#dataset.set_format("torch",columns=["id","input_values", "attention_mask", "label"])


test_dataloader = DataLoader(
    dataset["test"], batch_size=32, collate_fn=default_data_collator
)



for batch in test_dataloader:
#     print(batch.items())
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model_ft(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()

IndexError: Target 12 is out of bounds.